In [1]:
import sys, os
sys.path.insert(0, "..")

In [2]:
from ai_ben.train import get_batch
from chess import Chess
from ai_ben.ai import Agent, Plumbing
from ai_ben.model import TransformerModel
from ai_ben.config import Config

In [3]:
import pandas as pd

import random

from copy import deepcopy
from tqdm import tqdm

In [4]:
game_name="test"
epoch=0 
train=False
active_model='model-active.pth.tar'
new_model='model-new.pth.tar'
search_amount=50
max_depth=5
best_of=5

In [5]:
if (epoch+1) % best_of == 0:
    a_colour = random.choice(['w', 'b'])
elif (epoch+1) % 2 == 0:
    a_colour = 'b'
else:
    a_colour = 'w'

In [6]:
if a_colour == 'w':
    w_bot = deepcopy(Agent(search_amount=search_amount, max_depth=max_depth, train=train, model=active_model))
    b_bot = deepcopy(Agent(search_amount=search_amount, max_depth=max_depth, train=train, model=new_model))
elif a_colour == 'b':
    w_bot = deepcopy(Agent(search_amount=search_amount, max_depth=max_depth, train=train, model=new_model))
    b_bot = deepcopy(Agent(search_amount=search_amount, max_depth=max_depth, train=train, model=active_model))

In [7]:
log = []
plumbing = Plumbing()
chess_game = Chess()

In [16]:
chess_game.p_move

1

In [19]:
w_bot.choose_action(chess_game)

('', '')

In [28]:
w_bot.choose_action(Chess())

('G1', 'h3')

In [29]:
w_bot.log

[]

In [30]:
cur_pos

(3, 5)

In [ ]:
def make_move(bot, game):
    f, t = bot.choose_action(game)
    

In [8]:
pbar = tqdm()
while True:
    if chess_game.p_move == 1:
        cur,next = w_bot.choose_action(chess_game)
        w_log = pd.DataFrame(w_bot.log).drop_duplicates()
        if len(w_log) > 0:
            if 'imag_log' not in locals():
                imag_log = pd.DataFrame(columns=list(w_log.columns.values))
            for k,v in w_log.groupby(['value0','value1','value2']):
                t_log = pd.DataFrame(log)
                for i,x in enumerate(k):
                    t_log[f'value{i}'] = [x]*len(t_log)
                t_log = pd.concat([t_log, v], ignore_index=True)
                imag_log = pd.concat([imag_log, t_log], ignore_index = True) 
            imag_log = imag_log.drop_duplicates()
    else:
        cur,next = b_bot.choose_action(chess_game)
        b_log = pd.DataFrame(b_bot.log).drop_duplicates()
        if len(b_log) > 0:
            if 'imag_log' not in locals():
                imag_log = pd.DataFrame(columns=list(b_log.columns.values))
            for k,v in b_log.groupby(['value0','value1','value2']):
                t_log = pd.DataFrame(log)
                for i,x in enumerate(k):
                    t_log[f'value{i}'] = [x]*len(t_log)
                t_log = pd.concat([t_log, v], ignore_index=True)
                imag_log = pd.concat([imag_log, t_log], ignore_index = True) 
            imag_log = imag_log.drop_duplicates()
    #print(f'w {cur.lower()}-->{next.lower()} | EPOCH:{epoch} BOARD:{game_name} MOVE:{len(log)} HASH:{chess_game.EPD_hash()}\n') if chess_game.p_move > 0 else print(f'b {cur.lower()}-->{next.lower()} | EPOCH:{epoch} BOARD:{game_name} MOVE:{len(log)} HASH:{chess_game.EPD_hash()}\n')
    enc_game = plumbing.encode_state(chess_game)
    
    if chess_game.move(cur, next) == False:
        valid = False
    else:
        valid = True
        cur_pos = chess_game.board_2_array(cur)
        next_pos = chess_game.board_2_array(next)
        log.append({**{f'state{i}':float(s) for i, s in enumerate(enc_game[0])},
                    **{f'action{x}':1 if x == ((cur_pos[0]+(cur_pos[1]*8))*64)+(next_pos[0]+(next_pos[1]*8)) else 0 for x in range(4096)}})
    if (chess_game.p_move == 1) or (chess_game.p_move == -1):
        state = chess_game.check_state(chess_game.EPD_hash())
        if state == '50M' or state == '3F':
            state = [0, 1, 0] #Auto tie
        elif state == 'PP':
            chess_game.pawn_promotion(n_part='Q') #Auto queen
        if state != [0, 1, 0]:
            state = chess_game.is_end()
    else:
        state = chess_game.is_end()
        if state == [0, 0, 0]:
            if chess_game.check_state(chess_game.EPD_hash()) == 'PP':
                chess_game.pawn_promotion()
    if sum(state) > 0:
        #print(f'FINISHED | EPOCH:{epoch} BOARD:{game_name} MOVE:{len(log)} STATE:{state}\n')
        game_train_data = pd.DataFrame(log)
        for i, x in enumerate(state):
            game_train_data[f'value{i}'] = [x]*len(log)
        if 'imag_log' in locals():
            game_train_data = pd.concat([game_train_data, imag_log], ignore_index = True) 
        game_train_data = game_train_data.astype(float)
        break
    if valid == True:
        chess_game.p_move = chess_game.p_move * (-1)
    
    pbar.update()

pbar.close()

16it [02:33,  9.58s/it]


In [10]:
game_train_data.shape

(16, 4164)